**intialize the Spark Session**

In [ ]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder\
    .appName("Employee")\
    .getOrCreate()
)

**Load the Dataset**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
a= spark.read.csv("/content/drive/MyDrive/large_employee_dataset.csv", header=True, inferSchema=True)
a.show()

+----------+--------------------+---+----------+------+-----------+--------+-------------------+
|EmployeeID|                Name|Age|Department|Salary|JoiningDate|  Status|               City|
+----------+--------------------+---+----------+------+-----------+--------+-------------------+
|      4128|     Charles Johnson| 52|        HR| 64039| 2018-07-07|Resigned|          Allentown|
|      6094|       Dylan Camacho| 57| Marketing| 34686| 2015-08-25|  Active|        Anthonyfort|
|      5883| Mr. Ryan Bowman Jr.| 29|   Finance| 64541| 2025-03-11|On Leave|          Gilesstad|
|      9146|          Brian Ball| 24|     Sales| 87831| 2015-10-01|Resigned|       Jenniferfurt|
|      1918|       Angela Hooper| 26|   Finance|108773| 2019-08-14|On Leave|       Lake Amystad|
|      4600|Alexander Johnson...| 45|     Sales| 75671| 2016-04-21|On Leave|         Russohaven|
|      6253|         Steven Lane| 47|   Finance| 64982| 2021-07-25|  Active|        East Robert|
|      8278|       Riley Johns

**Basic Exploration**

In [9]:
#1.Show the top 10 rows
a.show(10)
#2.Count the total number of employees
a.count()
#3.Display unique departments
a.select("Department").distinct().show()

+----------+--------------------+---+----------+------+-----------+--------+------------+
|EmployeeID|                Name|Age|Department|Salary|JoiningDate|  Status|        City|
+----------+--------------------+---+----------+------+-----------+--------+------------+
|      4128|     Charles Johnson| 52|        HR| 64039| 2018-07-07|Resigned|   Allentown|
|      6094|       Dylan Camacho| 57| Marketing| 34686| 2015-08-25|  Active| Anthonyfort|
|      5883| Mr. Ryan Bowman Jr.| 29|   Finance| 64541| 2025-03-11|On Leave|   Gilesstad|
|      9146|          Brian Ball| 24|     Sales| 87831| 2015-10-01|Resigned|Jenniferfurt|
|      1918|       Angela Hooper| 26|   Finance|108773| 2019-08-14|On Leave|Lake Amystad|
|      4600|Alexander Johnson...| 45|     Sales| 75671| 2016-04-21|On Leave|  Russohaven|
|      6253|         Steven Lane| 47|   Finance| 64982| 2021-07-25|  Active| East Robert|
|      8278|       Riley Johnson| 49|        HR| 43449| 2015-08-03|Resigned|  New Thomas|
|      852

**Filtering & Sorting**

In [10]:
#4.Filter all employees in the "IT" department
a.filter(a.Department == "IT").show()
#5.Show employees aged between 30 and 40
a.filter((a.Age >= 30) & (a.Age <= 40)).show()
#6.Sort employees by salary in descending order
a.orderBy(a.Salary.desc()).show()

+----------+-------------------+---+----------+------+-----------+--------+------------------+
|EmployeeID|               Name|Age|Department|Salary|JoiningDate|  Status|              City|
+----------+-------------------+---+----------+------+-----------+--------+------------------+
|      6598|        Mary Henson| 58|        IT| 63951| 2021-08-25|  Active|       Port Tricia|
|      8518|   Elizabeth Abbott| 22|        IT| 91732| 2022-11-05|  Active|       Douglasside|
|      9506|        Thomas Dunn| 45|        IT| 90340| 2020-07-12|On Leave|    Lindseychester|
|      9663|        Glenn Mason| 43|        IT|109189| 2020-03-27|On Leave|      Katelynburgh|
|      2106|     Richard Bailey| 45|        IT| 30950| 2021-06-29|Resigned|        North John|
|      8212|      Jacob Jackson| 35|        IT| 54812| 2020-09-18|On Leave|South Veronicastad|
|      6354|     Nicole Gilmore| 35|        IT|104202| 2018-05-04|  Active|       East Joseph|
|      5716|         David Wang| 49|        IT| 94

**Aggregation Tasks**

In [11]:
#7Get the average salary by department
from pyspark.sql.functions import avg
a.groupBy("Department").agg(avg("Salary").alias("Average_Salary")).show()
#8.Count of employees by Status
a.groupBy("Status").count().show()
#9.Highest salary in each city
from pyspark.sql.functions import max
a.groupBy("City").agg(max("Salary").alias("Highest_Salary")).show()

+----------+-----------------+
|Department|   Average_Salary|
+----------+-----------------+
|     Sales|77488.54545454546|
|        HR|76091.27450980392|
|   Finance|72834.75630252101|
| Marketing| 71958.1888888889|
|        IT|73116.25555555556|
+----------+-----------------+

+--------+-----+
|  Status|count|
+--------+-----+
|Resigned|  159|
|  Active|  172|
|On Leave|  169|
+--------+-----+

+----------------+--------------+
|            City|Highest_Salary|
+----------------+--------------+
|   Wilsonchester|         67025|
|     Bradshawton|        111116|
|       Steventon|         32009|
|     Lake Alyssa|         84903|
|      North Lisa|         57898|
|    North Marvin|         66252|
|     Jenniferton|         39907|
|     Buckleyview|         50109|
|     Burtonville|         98492|
|    Johnsonmouth|         48799|
|    South Joseph|         52456|
|  Lindseychester|         90340|
|   North Stephen|         91947|
|Port Nicoleshire|         57537|
|    Jerrychester|    

**GroupBy and Analysis**

In [15]:
#10.Total number of employees who joined each year
from pyspark.sql.functions import year
a.withColumn("JoinYear", year("JoiningDate"))\
  .groupBy("JoinYear")\
  .count()\
  .show()
#11.Department-wise count of employees who are currently "Active"
a.filter(a.Status == "Active")\
  .groupBy("Department")\
  .count()\
  .show()
#12.Average age of employees per department
a.groupBy("Department")\
  .agg(avg("Age").alias("Average_Age"))\
  .show()


+--------+-----+
|JoinYear|count|
+--------+-----+
|    2025|   27|
|    2018|   52|
|    2015|   37|
|    2023|   47|
|    2022|   49|
|    2019|   52|
|    2020|   56|
|    2016|   49|
|    2024|   38|
|    2017|   44|
|    2021|   49|
+--------+-----+

+----------+-----+
|Department|count|
+----------+-----+
|     Sales|   32|
|        HR|   37|
|   Finance|   45|
| Marketing|   32|
|        IT|   26|
+----------+-----+

+----------+------------------+
|Department|       Average_Age|
+----------+------------------+
|     Sales|40.535353535353536|
|        HR| 41.46078431372549|
|   Finance| 39.21008403361345|
| Marketing| 41.82222222222222|
|        IT| 38.68888888888889|
+----------+------------------+



**Joining**

In [16]:
#13.Create another dataset with City and Region, and join it
# Sample city-region mapping
from pyspark.sql import Row
region_data = [
    Row(City="New York", Region="East"),
    Row(City="San Francisco", Region="West"),
    Row(City="Chicago", Region="Midwest"),
    Row(City="Los Angeles", Region="West"),
    Row(City="Houston", Region="South"),
    Row(City="Miami", Region="South"),
    Row(City="Seattle", Region="West"),
    Row(City="Boston", Region="East"),
    Row(City="Dallas", Region="South"),
    Row(City="Philadelphia", Region="East"),
    Row(City="Phoenix", Region="West"),
    Row(City="San Diego", Region="West"),
    Row(City="Denver", Region="Midwest"),
    Row(City="San Antonio", Region="South"),
    Row(City="San Jose", Region="West"),
    Row(City="Atlanta", Region="South"),
    Row(City="Detroit", Region="Midwest"),
    Row(City="Portland", Region="West"),
    Row(City="Minneapolis", Region="Midwest"),
    Row(City="Orlando", Region="South"),
    Row(City="Tampa", Region="South"),
    Row(City="Nashville", Region="Midwest"),
    Row(City="Sacramento", Region="West"),
    Row(City="Kansas City", Region="Midwest"),
    Row(City="Charlotte", Region="South"),
]

region_df = spark.createDataFrame(region_data)
joined_df = a.join(region_df, on="City", how="left")
joined_df.show()


+-------------------+----------+--------------------+---+----------+------+-----------+--------+------+
|               City|EmployeeID|                Name|Age|Department|Salary|JoiningDate|  Status|Region|
+-------------------+----------+--------------------+---+----------+------+-----------+--------+------+
|        East Robert|      6253|         Steven Lane| 47|   Finance| 64982| 2021-07-25|  Active|  NULL|
|          Allentown|      4128|     Charles Johnson| 52|        HR| 64039| 2018-07-07|Resigned|  NULL|
|         Tonyamouth|      6406|       Patrick Chung| 27|        HR|116423| 2024-07-05|  Active|  NULL|
|       Jenniferfurt|      9146|          Brian Ball| 24|     Sales| 87831| 2015-10-01|Resigned|  NULL|
|     North Brittany|      8989|       Scott Burnett| 48|     Sales| 93690| 2016-04-25|Resigned|  NULL|
|          Gilesstad|      5883| Mr. Ryan Bowman Jr.| 29|   Finance| 64541| 2025-03-11|On Leave|  NULL|
|          Port Mark|      3326|       Michael Brown| 28|       

In [17]:
#14.Group salaries by Region after the join
joined_df.groupBy("Region")\
  .agg(avg("Salary").alias("Average_Salary"))\
  .show()

+------+--------------+
|Region|Average_Salary|
+------+--------------+
|  NULL|     74313.424|
+------+--------------+



**Date Operations**

In [20]:
#15.Calculate years of experience for each employee
from pyspark.sql.functions import current_date, datediff, col,round
exp = a.withColumn("Experience_Years", round(datediff(current_date(), a.JoiningDate)/365, 2))
exp.select("EmployeeID", "Name", "Experience_Years").show()
#16.List all employees with more than 5 years of experience
exp.filter(exp.Experience_Years > 5).show()

+----------+--------------------+----------------+
|EmployeeID|                Name|Experience_Years|
+----------+--------------------+----------------+
|      4128|     Charles Johnson|            6.91|
|      6094|       Dylan Camacho|            9.78|
|      5883| Mr. Ryan Bowman Jr.|            0.23|
|      9146|          Brian Ball|            9.68|
|      1918|       Angela Hooper|            5.81|
|      4600|Alexander Johnson...|            9.12|
|      6253|         Steven Lane|            3.86|
|      8278|       Riley Johnson|            9.84|
|      8520|    Emily Washington|            3.51|
|      1298|     Valerie Fleming|            5.49|
|      5157|     Tracy Hughes MD|            5.01|
|      7403|    Johnathan Harmon|            4.24|
|      3326|       Michael Brown|            1.62|
|      8989|       Scott Burnett|            9.11|
|      4676|  Christopher Fuller|             4.1|
|      6598|         Mary Henson|            3.78|
|      4136|       Jerome Torre